In [ ]:
#import numpy as np
#import scipy as sp
#import matplotlib as mpl
#import matplotlib.cm as cm
#import matplotlib.pyplot as plt
#import pandas as pd
#import time
#pd.set_option('display.width', 500)
#pd.set_option('display.max_columns', 100)
#pd.set_option('display.notebook_repr_html', True)

In [1]:
#import re, collections
import re

In [2]:
max_edit_distance = 2
dictionary = {}
longest_word_length = 0

In [3]:
def get_deletes_list(w):
    '''given a word, derive strings with up to max_edit_distance characters deleted'''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

In [4]:
# get_deletes_list("tomorrow")

In [5]:
def create_dictionary_entry(w):
    '''add word and its derived deletions to dictionary'''
    # check if word is already in dictionary
    # dictionary entries are in the form: (list of suggested corrections, frequency of word in corpus)
    global longest_word_length
    new_real_word_added = False
    if w in dictionary:
        dictionary[w] = (dictionary[w][0], dictionary[w][1] + 1)  # increment count of word in corpus
    else:
        dictionary[w] = ([], 1)  
        longest_word_length = max(longest_word_length, len(w))
        
    if dictionary[w][1]==1:
        # first appearance of word in corpus
        # n.b. word may already be in dictionary as a derived word (deleting character from a real word)
        # but counter of frequency of word in corpus is not incremented in those cases)
        new_real_word_added = True
        deletes = get_deletes_list(w)
        for item in deletes:
            if item in dictionary:
                # add (correct) word to delete's suggested correction list if not already there
                if item not in dictionary[item][0]:
                    dictionary[item][0].append(w)
            else:
                dictionary[item] = ([w], 0)  # note frequency of word in corpus is not incremented
        
    return new_real_word_added

In [6]:
def create_dictionary(fname):
    with open(fname) as file:
        word_count = 0
        print "Creating dictionary..." 
        
        for line in file:
            words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for word in words:
                if create_dictionary_entry(word):
                    word_count += 1
                    
        print ("total unique words in corpus: %i" % word_count)
        print ("total items in dictionary (corpus words and deletions): %i" % len(dictionary))
        print ("  edit distance for deletions: %i" % max_edit_distance)
        print ("  length of longest word in corpus: %i" % longest_word_length)
        
        return dictionary

In [7]:
%time
create_dictionary("/Users/K-Lo/Desktop/smaller.txt")
pass

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
Creating dictionary...
total unique words in corpus: 12020
total items in dictionary (corpus words and deletions): 323605
  edit distance for deletions: 2
  length of longest word in corpus: 18


In [68]:
dictionary["bar"]

(['baker',
  'bear',
  'board',
  'bears',
  'bark',
  'beard',
  'bars',
  'bare',
  'briar',
  'barre',
  'barry',
  'baron'],
 14)

In [8]:
# dictionary

{'pinciples': (['principles'], 0),
 'samul': (['samuel'], 0),
 'fahoned': (['fashioned'], 0),
 'ecipients': (['recipients'], 0),
 'woodd': (['wooded'], 0),
 'woode': (['wooden', 'wooded'], 0),
 'prosprous': (['prosperous'], 0),
 'woodn': (['wooden'], 0),
 'woodo': (['woodrow'], 0),
 'commdties': (['commodities'], 0),
 'woodr': (['woodrow'], 0),
 'woods': ([], 7),
 'sunde': (['sounded'], 0),
 'intgue': (['intrigue'], 0),
 'quesins': (['questions'], 0),
 'deerous': (['dexterous'], 0),
 'cmlimentary': (['complimentary'], 0),
 'sterity': (['posterity'], 0),
 'insensiblty': (['insensibility'], 0),
 'atenation': (['alternation'], 0),
 'emptie': (['emptied'], 0),
 'canes': (['chances', 'changes', 'candles'], 0),
 'canet': (['cabinet'], 0),
 'fundatios': (['foundations'], 0),
 'uccesor': (['successor'], 0),
 'erfectly': (['perfectly'], 0),
 'acurately': (['accurately'], 0),
 'nvaribly': (['invariably'], 0),
 'absrbd': (['absorbed'], 0),
 'absrbe': (['absorbed'], 0),
 'nvarible': (['invariable'

In [69]:
def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance between sequences.

    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

In [74]:
def get_suggestions(string):
    '''return list of suggested corrections for potentially incorrectly spelled word'''
    if (len(string) - longest_word_length) > max_edit_distance:
        print "no items in dictionary within maximum edit distance"
        return []
    
    # suggestions = []
    # s_dictionary = {}
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        # print "processing '%s'" % q_item
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus, and not already in suggestion list
            # so add to suggestion dictionary, indexed by the word with value (frequency in corpus, edit distance)
            # note q_items that are not the input string are shorter than input string 
            # since only deletes are added (unless manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = (dictionary[q_item][1], len(string) - len(q_item))
            
            ## the suggested corrections for q_item as stored in dictionary (whether or not
            ## q_item itself is a valid word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    # compute edit distance
                    # if len(sc_item)==len(q_item):
                    #    item_dist = len(string) - len(q_item)
                    # suggested items should always be longer (unless manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)
                    #elif len(q_item)==len(string):
                        # a suggestion could be longer or shorter than original string (bug in original FAROO?)
                        # if suggestion is from string's suggestion list, sc_item will be longer
                        # if suggestion is from a delete's suggestion list, sc_item may be shorter
                    #   item_dist = abs(len(sc_item) - len(q_item))
                    #else:
                    # check in original code, but probably not necessary because string has already checked
                    assert sc_item!=string
                    
                    # calculate edit distance using, for example, Damerau-Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    if item_dist<=max_edit_distance:
                        assert sc_item in dictionary  # should already be in dictionary if in suggestion list
                        suggest_dict[sc_item] = (dictionary[sc_item][1], item_dist)
        
        # now generate deletes (e.g. a substring of string or of a delete) from the queue item
        # as additional items to check -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this
             
    # queue is now empty: convert suggestions in dictionary to list for output
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested corrections only:
    # return sorted(suggest_dict, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    as_list = suggest_dict.items()
    return sorted(as_list, key = lambda (term, (freq, dist)): (dist, -freq))

    '''
    Option 1:
    get_suggestions("file")
    ['file', 'five', 'fire', 'fine', ...]
    
    Option 2:
    get_suggestions("file")
    [('file', (5, 0)),
     ('five', (67, 1)),
     ('fire', (54, 1)),
     ('fine', (17, 1))...]  
    '''

In [79]:
%time
get_suggestions("whom")

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


[('whom', (51, 0)),
 ('who', (461, 1)),
 ('whim', (2, 1)),
 ('whoa', (2, 1)),
 ('from', (849, 2)),
 ('him', (486, 2)),
 ('what', (456, 2)),
 ('when', (443, 2)),
 ('them', (312, 2)),
 ('how', (197, 2)),
 ('room', (172, 2)),
 ('why', (98, 2)),
 ('whole', (81, 2)),
 ('home', (77, 2)),
 ('whose', (36, 2)),
 ('show', (29, 2)),
 ('won', (24, 2)),
 ('shot', (16, 2)),
 ('wood', (13, 2)),
 ('hot', (12, 2)),
 ('hum', (9, 2)),
 ('shop', (6, 2)),
 ('whip', (4, 2)),
 ('warm', (4, 2)),
 ('whig', (3, 2)),
 ('com', (2, 2)),
 ('htm', (1, 2)),
 ('shoe', (1, 2)),
 ('whit', (1, 2)),
 ('whims', (1, 2)),
 ('whoso', (1, 2)),
 ('wool', (1, 2)),
 ('doom', (1, 2)),
 ('worm', (1, 2))]